### 1. 读取数据
读取时加上names

In [1]:
import pandas as pd
import numpy as np

In [2]:
column_name =[
    "Sample code number","Clump Thickness","Uniformity of Cell Size",
    "Uniformity of Cell Shape","Marginal Adhesion","Single Epithelial Cell Size",
    "Bare Nuclei","Bland Chromatin","Normal Nucleoli","Mitoses","Class"
    ]
data = pd.read_csv("../../../datas/breast_cancer/breast-cancer-wisconsin.data",names=column_name)
data

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


### 2. 数据处理  
缺失值处理

In [3]:
# 替换缺失值
data = data.replace(to_replace='?',value=np.nan)
# 删除缺失样本
data.dropna(inplace=True)
data.isnull().any()

Sample code number             False
Clump Thickness                False
Uniformity of Cell Size        False
Uniformity of Cell Shape       False
Marginal Adhesion              False
Single Epithelial Cell Size    False
Bare Nuclei                    False
Bland Chromatin                False
Normal Nucleoli                False
Mitoses                        False
Class                          False
dtype: bool

### 3. 数据集划分

In [4]:
# 筛选目标值与特征值
x = data.iloc[:,1:-1]
y = data["Class"]

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=8)

### 4. 标准化

In [6]:
from sklearn.preprocessing import StandardScaler
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

### 5. 预估器

In [7]:
from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()
estimator.fit(x_train,y_train)

LogisticRegression()

### 6. 模型评估
estimator.coef_

In [8]:
estimator.intercept_

array([-1.22004736])

### 6. 模型评估

In [9]:
# 法一 直接比对真实值和预测值
y_predict =  estimator.predict(x_test)
print("y_predict:\n",y_predict)
print("直接比对真实值和预测值：\n",y_predict == y_test)

# 法二 计算准确率
score = estimator.score(x_test,y_test)
print("准确率：\n",score)

y_predict:
 [4 2 2 4 2 2 2 2 2 4 4 2 2 4 2 2 2 2 2 2 2 4 2 4 2 2 4 2 2 2 2 2 4 2 2 4 2
 4 2 4 4 2 4 2 2 4 4 4 2 2 2 4 2 2 4 4 2 4 4 2 2 4 2 2 2 4 2 2 2 2 2 2 2 2
 2 4 2 4 4 4 4 2 4 2 4 2 2 2 4 4 2 2 4 4 2 2 2 4 2 4 2 2 4 2 4 4 2 2 4 2 2
 2 2 2 4 2 4 4 4 2 2 4 2 2 2 4 4 2 2 2 2 2 4 4 4 2 2 4 2 4 4 2 2 4 2 4 4 2
 2 4 4 2 4 4 4 2 2 4 2 2 2 2 2 2 2 4 4 2 2 2 4]
直接比对真实值和预测值：
 230    True
639    True
78     True
214    True
685    True
       ... 
122    True
541    True
163    True
496    True
229    True
Name: Class, Length: 171, dtype: bool
准确率：
 0.9707602339181286


### 分类的评估方法

#### 1. 精确率与召回率
* 精确率：预测结果正例样本中真实为正例的比例
* 召回率：真实结果正例样本中预测为正例的比例
* F1-Score：反映模型的稳健性(越大时说明精确率与召回率高)  
**一般主要看召回率**


In [10]:
from sklearn.metrics import classification_report
report = classification_report(y_test,y_predict,labels=[2,4],target_names = ["良性","恶性"])
print(report)

              precision    recall  f1-score   support

          良性       0.97      0.98      0.98       104
          恶性       0.97      0.96      0.96        67

    accuracy                           0.97       171
   macro avg       0.97      0.97      0.97       171
weighted avg       0.97      0.97      0.97       171




#### 2.ROC曲线与AUC指标
**AUC最小值为0.5，最大值为1，越大说明分类器越好**  
召回率和精确率会收到样本不均衡影响  
样本不均衡时，AUC会很低  

In [11]:
# y_true必须 为0(反例),1(正例)
y_true = np.where(y_test>3,1,0)

In [12]:
y_true

array([1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1])

In [13]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true,y_predict)

0.9679965556831229

### 7. 模型保存与加载


In [14]:
import joblib

In [15]:
joblib.dump(estimator,"test.pkl")

['test.pkl']

In [16]:
estimator = joblib.load("test.pkl")
estimator

LogisticRegression()